In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#df = pd.read_csv('/content/drive/My Drive/Data/datasets_20710_26737_Bengaluru_House_Data.csv')
df = pd.read_csv('../Data/datasets_20710_26737_Bengaluru_House_Data.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
# Data cleaning
df1 = df.drop(['area_type','society','balcony','availability'],axis='columns')
df1.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [4]:
df1.isnull().sum()
df2=df1.dropna()

In [5]:
df2['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [6]:
df2['bhk']= df2['size'].apply(lambda x: x.split(' ')[0])
df3 = df2.drop(['size'],axis = 'columns')
df3.tail()

C:\Users\Ari\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,location,total_sqft,bath,price,bhk
13315,Whitefield,3453,4.0,231.0,5
13316,Richards Town,3600,5.0,400.0,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.0,2
13318,Padmanabhanagar,4689,4.0,488.0,4
13319,Doddathoguru,550,1.0,17.0,1


In [7]:
df3['bhk'].value_counts()


2     5527
3     4832
4     1395
1      649
5      353
6      221
7      100
8       89
9       54
10      14
11       4
13       1
43       1
14       1
27       1
12       1
19       1
16       1
18       1
Name: bhk, dtype: int64

In [8]:
def checkFloat(x):
    try:
        float(x)
    except:
        return False
    return True
        
df3[~df3.total_sqft.apply(checkFloat)].head()

,location,total_sqft,bath,price,bhk
30,Yelahanka,2100 - 2850,4.0,186.000,4
122,Hebbal,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,1042 - 1105,2.0,54.005,2
165,Sarjapur,1145 - 1340,2.0,43.490,2
188,KR Puram,1015 - 1540,2.0,56.800,2


In [9]:
# Not going to tackle the unit conversions, however will fix the range problem by taking avg of two
def isSqft(x):
    try:
        if len(x.split('-'))>1:
            return float(x.split('-')[0]) + float(x.split('-')[1])/2
        else:
            return float(x)
    except:
        return None       


In [10]:
#df3['total_sqft']=df3.total_sqft.apply(lambda x: (float(x.split('-')[0]) + float(x.split('-')[1])/2) if len(x.split('-'))>1 else x)
df4 = df3.copy()
df4['total_sqft'] = df4.total_sqft.apply(isSqft)

In [11]:
# Feature Engineering

# Calculdate price/sqft
df5 =  df4.copy()
df5['price_sqft'] = df4['price']*100000/df4['total_sqft']
df5['price_sqft'] = df5['price_sqft'].astype(float)

In [12]:
df5.head()

,location,total_sqft,bath,price,bhk,price_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000


In [13]:
df5['location'] = df5['location'].apply(lambda x: x.strip())
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending =False)
location_stats[location_stats<=10]

location
BTM 1st Stage                                   10
Basapura                                        10
Sector 1 HSR Layout                             10
Naganathapura                                   10
Kalkere                                         10
Nagadevanahalli                                 10
Nagappa Reddy Layout                            10
Sadashiva Nagar                                 10
Gunjur Palya                                    10
Dairy Circle                                    10
Ganga Nagar                                     10
Dodsworth Layout                                10
1st Block Koramangala                           10
Chandra Layout                                   9
Jakkur Plantation                                9
2nd Phase JP Nagar                               9
Yemlur                                           9
Mathikere                                        9
Medahalli                                        9
Volagerekallahalli    

In [14]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats[location_stats<=10] else x)

In [15]:
len(df5.location.unique()) # This should help when we are gonna apply Onehotencoding

242

In [17]:
df5

,location,total_sqft,bath,price,bhk,price_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,other,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


In [16]:
# Remove outlier
df5.total_sqft = df5.total_sqft.astype(float)
df5.bhk = df5.bhk.astype(float)
df6 = df5[~(df5.total_sqft/df5.bhk<300)]

In [17]:
# Check for the outlier, and removing using z score
df6.price_sqft.describe()
def removeOutlier(df_arg):
    df_out = pd.DataFrame()
    for key, subdf in df_arg.groupby('location'):
        m = np.mean(subdf.price_sqft)
        sd = np.std(subdf.price_sqft)
        reduced_df = subdf[(subdf.price_sqft > (m-sd)) & (subdf.price_sqft <= (m+sd))]
        df_out = pd.concat([df_out,reduced_df], ignore_index=True)
    return df_out

In [18]:
df7 = removeOutlier(df6)

In [19]:
df8 = df7.drop(['price_sqft'], axis= 'columns')

In [20]:
#Using oneHotEncoding
dummies = pd.get_dummies(df8.location)
df9 = pd.concat([df8.drop(['location'], axis='columns'), dummies.drop(['other'],axis = 'columns')],axis = 'columns')
df9.head()

,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,428.0,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,194.0,3.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,235.0,3.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1200.0,2.0,130.0,3.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1235.0,2.0,148.0,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# Model building

X= df9.drop(['price'],axis='columns')
Y = df9.price

In [22]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = .2, random_state = 10)   


In [23]:
#Use linear regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,Y_train)
lr.score(X_test, Y_test)

0.844613795737013

In [24]:
#Using k-flod cross validation

from sklearn.model_selection import ShuffleSplit, cross_val_score, StratifiedKFold

cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
cross_val_score(LinearRegression(), X,Y, cv=cv)

array([0.72343917, 0.86729659, 0.79579114, 0.82441973, 0.80496198,
       0.85445537, 0.8503268 , 0.80721877, 0.75016849, 0.75648308])

# Hyperparameter tuning and selecting best model for regression

In [44]:
# Regression libs

from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

def find_best_model(X = None,Y= None):
    algos={
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
            },
          'lasso': {
              'model': Lasso(),
              'params': {
                  'alpha': [1,2],
                  'selection': ['random', 'cyclic']
              }
          },
          'decision_tree': {
              'model': DecisionTreeRegressor(),
              'params':{
                  'criterion': ['mse', 'friedman_mse'],
                  'splitter': ['best','random']
              }
          },
        'Knn_regression': {
            'model': KNeighborsRegressor(),
            'params':{
                'n_neighbors': [2,3,4,5,6]
            }
        }
    }
    scores=[]
    cv = ShuffleSplit(n_splits = 50, test_size=.2, random_state = 0)
    for algo, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score = False)
        gs.fit(X,Y)
        scores.append({
            'model':algo,
            'best_score':gs.best_score_,
            'best_param':gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_score','best_param'])

In [45]:
 find_best_model(X,Y)

,model,best_score,best_param
0,linear_regression,0.798462,{'normalize': True}
1,lasso,0.671633,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.654814,"{'criterion': 'friedman_mse', 'splitter': 'ran..."
3,Knn_regression,0.632782,{'n_neighbors': 6}


In [46]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr.predict([x])[0]

predict_price('1st Phase JP Nagar',1000, 2, 3)

81.07433227331875

# In order to use the prediction in Realtime, below steps help to use the model
##Saving the model in a pickle
##Take the proper columns

In [30]:
import pickle
with open('bangalore home price model.pickle','wb') as f:
    pickle.dump(lr,f)

In [31]:
# Taking the right cols
import json
cols = {
    'data_cols': [col.lower() for col in X.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(cols))